# Part 1A: Initial Data Inspection

## Objective
Create a comprehensive inspection of the Premier League analytics database, examining both current snapshot data (is_current=true) and full historical records. Identify data structure, quality issues, and temporal coverage to inform machine learning model development.

## Database Information
- **Database:** `data/premierleague_analytics.duckdb` (DuckDB format)
- **Tables:** analytics_players, analytics_keepers, analytics_squads, analytics_fixtures
- **SCD Type 2 tracking** with is_current flag
- **Team-specific gameweek tracking** (teams can be at different gameweeks)

---

# Section 1: Setup and Imports

In [ ]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Database connection
db_path = "../../../data/premierleague_analytics.duckdb"
conn = duckdb.connect(db_path, read_only=True)

print("✅ Connected to analytics database")
print(f"📊 Database path: {db_path}")
print(f"⏰ Analysis started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Section 2: Table-Level Overview

Examining the structure and basic statistics for all four tables in the database.

## 2.1 Analytics Players Table

In [ ]:
# Get table structure
print("TABLE STRUCTURE: analytics_players")
print("=" * 100)
table_info_players = conn.execute("PRAGMA table_info(analytics_players)").fetchdf()
print(table_info_players.to_string(index=False))
print("\n")

# Get row counts
total_rows_players = conn.execute("SELECT COUNT(*) FROM analytics_players").fetchone()[0]
current_rows_players = conn.execute("SELECT COUNT(*) FROM analytics_players WHERE is_current = true").fetchone()[0]

print(f"Total historical records: {total_rows_players:,}")
print(f"Current records (is_current=true): {current_rows_players:,}")
print(f"Percentage of current: {(current_rows_players/total_rows_players)*100:.1f}%")

In [ ]:
# Show 5 sample current records
print("SAMPLE CURRENT RECORDS (is_current = true):")
print("=" * 100)
sample_current_players = conn.execute("""
    SELECT * FROM analytics_players 
    WHERE is_current = true 
    LIMIT 5
""").fetchdf()
display(sample_current_players)

print("\n" + "=" * 100)
print("SAMPLE HISTORICAL RECORDS (is_current = false):")
print("=" * 100)
sample_historical_players = conn.execute("""
    SELECT * FROM analytics_players 
    WHERE is_current = false 
    LIMIT 5
""").fetchdf()
display(sample_historical_players)

## 2.2 Analytics Keepers Table

In [ ]:
# Get table structure
print("TABLE STRUCTURE: analytics_keepers")
print("=" * 100)
table_info_keepers = conn.execute("PRAGMA table_info(analytics_keepers)").fetchdf()
print(table_info_keepers.to_string(index=False))
print("\n")

# Get row counts
total_rows_keepers = conn.execute("SELECT COUNT(*) FROM analytics_keepers").fetchone()[0]
current_rows_keepers = conn.execute("SELECT COUNT(*) FROM analytics_keepers WHERE is_current = true").fetchone()[0]

print(f"Total historical records: {total_rows_keepers:,}")
print(f"Current records (is_current=true): {current_rows_keepers:,}")
print(f"Percentage of current: {(current_rows_keepers/total_rows_keepers)*100:.1f}%")

In [ ]:
# Show 5 sample current records
print("SAMPLE CURRENT RECORDS (is_current = true):")
print("=" * 100)
sample_current_keepers = conn.execute("""
    SELECT * FROM analytics_keepers 
    WHERE is_current = true 
    LIMIT 5
""").fetchdf()
display(sample_current_keepers)

print("\n" + "=" * 100)
print("SAMPLE HISTORICAL RECORDS (is_current = false):")
print("=" * 100)
sample_historical_keepers = conn.execute("""
    SELECT * FROM analytics_keepers 
    WHERE is_current = false 
    LIMIT 5
""").fetchdf()
display(sample_historical_keepers)

## 2.3 Analytics Squads Table

In [ ]:
# Get table structure
print("TABLE STRUCTURE: analytics_squads")
print("=" * 100)
table_info_squads = conn.execute("PRAGMA table_info(analytics_squads)").fetchdf()
print(table_info_squads.to_string(index=False))
print("\n")

# Get row counts
total_rows_squads = conn.execute("SELECT COUNT(*) FROM analytics_squads").fetchone()[0]
current_rows_squads = conn.execute("SELECT COUNT(*) FROM analytics_squads WHERE is_current = true").fetchone()[0]

print(f"Total historical records: {total_rows_squads:,}")
print(f"Current records (is_current=true): {current_rows_squads:,}")
print(f"Percentage of current: {(current_rows_squads/total_rows_squads)*100:.1f}%")

In [ ]:
# Show 5 sample current records
print("SAMPLE CURRENT RECORDS (is_current = true):")
print("=" * 100)
sample_current_squads = conn.execute("""
    SELECT * FROM analytics_squads 
    WHERE is_current = true 
    LIMIT 5
""").fetchdf()
display(sample_current_squads)

print("\n" + "=" * 100)
print("SAMPLE HISTORICAL RECORDS (is_current = false):")
print("=" * 100)
sample_historical_squads = conn.execute("""
    SELECT * FROM analytics_squads 
    WHERE is_current = false 
    LIMIT 5
""").fetchdf()
display(sample_historical_squads)

## 2.4 Analytics Fixtures Table

In [ ]:
# Get table structure
print("TABLE STRUCTURE: analytics_fixtures")
print("=" * 100)
table_info_fixtures = conn.execute("PRAGMA table_info(analytics_fixtures)").fetchdf()
print(table_info_fixtures.to_string(index=False))
print("\n")

# Get row counts
total_rows_fixtures = conn.execute("SELECT COUNT(*) FROM analytics_fixtures").fetchone()[0]

print(f"Total fixture records: {total_rows_fixtures:,}")

In [ ]:
# Show 5 sample records
print("SAMPLE FIXTURE RECORDS:")
print("=" * 100)
sample_fixtures = conn.execute("""
    SELECT * FROM analytics_fixtures 
    LIMIT 5
""").fetchdf()
display(sample_fixtures)

# Section 3: CURRENT DATA INSPECTION (is_current = true)

Focus on the most recent snapshot of data.

## 3.1 Record Counts by Entity

In [ ]:
# Players by position
print("PLAYERS BY POSITION (Current):")
print("=" * 80)
position_counts = conn.execute("""
    SELECT position, COUNT(*) as count
    FROM analytics_players
    WHERE is_current = true
    GROUP BY position
    ORDER BY count DESC
""").fetchdf()
print(position_counts.to_string(index=False))
print(f"\nTotal outfield players: {position_counts['count'].sum():,}")

# Keepers count
keeper_count = conn.execute("""
    SELECT COUNT(*) as count
    FROM analytics_keepers
    WHERE is_current = true
""").fetchone()[0]
print(f"Total keepers: {keeper_count:,}")

# Squads count
squad_count = conn.execute("""
    SELECT COUNT(*) as count
    FROM analytics_squads
    WHERE is_current = true
""").fetchone()[0]
print(f"Total squads: {squad_count:,}")

# Fixtures breakdown
print("\n" + "=" * 80)
print("FIXTURES BREAKDOWN:")
print("=" * 80)
fixtures_breakdown = conn.execute("""
    SELECT 
        is_completed,
        COUNT(*) as count
    FROM analytics_fixtures
    GROUP BY is_completed
""").fetchdf()
print(fixtures_breakdown.to_string(index=False))
print(f"\nTotal fixtures: {fixtures_breakdown['count'].sum():,}")

## 3.2 Gameweek Distribution (Current)

In [ ]:
# Gameweek distribution across squads
print("GAMEWEEK DISTRIBUTION ACROSS SQUADS (Current):")
print("=" * 100)
gw_distribution_squads = conn.execute("""
    SELECT 
        gameweek,
        COUNT(*) as num_squads,
        GROUP_CONCAT(squad_name, ', ') as squads
    FROM analytics_squads
    WHERE is_current = true
    GROUP BY gameweek
    ORDER BY gameweek
""").fetchdf()
print(gw_distribution_squads.to_string(index=False))

# Visualize gameweek distribution
plt.figure(figsize=(10, 6))
plt.bar(gw_distribution_squads['gameweek'], gw_distribution_squads['num_squads'], color='#1f77b4')
plt.xlabel('Gameweek', fontsize=12)
plt.ylabel('Number of Squads', fontsize=12)
plt.title('Current Gameweek Distribution Across Squads', fontsize=14, fontweight='bold')
plt.xticks(gw_distribution_squads['gameweek'])
plt.grid(axis='y', alpha=0.3)

# Save to outputs
output_dir = Path("../../outputs/01_data_inspection")
output_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(output_dir / "current_gameweek_distribution.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Chart saved to: {output_dir / 'current_gameweek_distribution.png'}")

## 3.3 Missing Data Analysis (Current)

In [ ]:
# For analytics_players - check all numeric columns for nulls
print("MISSING DATA ANALYSIS - ANALYTICS_PLAYERS (Current):")
print("=" * 100)

null_counts_players = conn.execute("""
    SELECT 
        COUNT(*) as total_records,
        SUM(CASE WHEN goals IS NULL THEN 1 ELSE 0 END) as null_goals,
        SUM(CASE WHEN assists IS NULL THEN 1 ELSE 0 END) as null_assists,
        SUM(CASE WHEN minutes_played IS NULL THEN 1 ELSE 0 END) as null_minutes,
        SUM(CASE WHEN matches_played IS NULL THEN 1 ELSE 0 END) as null_matches,
        SUM(CASE WHEN shots IS NULL THEN 1 ELSE 0 END) as null_shots,
        SUM(CASE WHEN passes_completed IS NULL THEN 1 ELSE 0 END) as null_passes_completed,
        SUM(CASE WHEN pass_completion_rate IS NULL THEN 1 ELSE 0 END) as null_pass_completion_rate,
        SUM(CASE WHEN tackles IS NULL THEN 1 ELSE 0 END) as null_tackles,
        SUM(CASE WHEN interceptions IS NULL THEN 1 ELSE 0 END) as null_interceptions
    FROM analytics_players
    WHERE is_current = true
""").fetchdf()

# Calculate percentages
total = null_counts_players['total_records'].iloc[0]
null_pct_goals = (null_counts_players['null_goals'].iloc[0] / total) * 100
null_pct_assists = (null_counts_players['null_assists'].iloc[0] / total) * 100
null_pct_minutes = (null_counts_players['null_minutes'].iloc[0] / total) * 100
null_pct_matches = (null_counts_players['null_matches'].iloc[0] / total) * 100
null_pct_shots = (null_counts_players['null_shots'].iloc[0] / total) * 100
null_pct_passes = (null_counts_players['null_passes_completed'].iloc[0] / total) * 100
null_pct_pass_rate = (null_counts_players['null_pass_completion_rate'].iloc[0] / total) * 100
null_pct_tackles = (null_counts_players['null_tackles'].iloc[0] / total) * 100
null_pct_interceptions = (null_counts_players['null_interceptions'].iloc[0] / total) * 100

missing_summary = pd.DataFrame({
    'Column': ['goals', 'assists', 'minutes_played', 'matches_played', 'shots', 
               'passes_completed', 'pass_completion_rate', 'tackles', 'interceptions'],
    'Null Count': [
        null_counts_players['null_goals'].iloc[0],
        null_counts_players['null_assists'].iloc[0],
        null_counts_players['null_minutes'].iloc[0],
        null_counts_players['null_matches'].iloc[0],
        null_counts_players['null_shots'].iloc[0],
        null_counts_players['null_passes_completed'].iloc[0],
        null_counts_players['null_pass_completion_rate'].iloc[0],
        null_counts_players['null_tackles'].iloc[0],
        null_counts_players['null_interceptions'].iloc[0]
    ],
    'Missing %': [null_pct_goals, null_pct_assists, null_pct_minutes, null_pct_matches,
                  null_pct_shots, null_pct_passes, null_pct_pass_rate, null_pct_tackles,
                  null_pct_interceptions]
})

print(missing_summary.to_string(index=False))

# Identify problematic columns (>5% missing)
problematic = missing_summary[missing_summary['Missing %'] > 5]
if len(problematic) > 0:
    print(f"\n⚠️  WARNING: {len(problematic)} columns have >5% missing data:")
    print(problematic[['Column', 'Missing %']].to_string(index=False))
else:
    print("\n✅ All columns have <5% missing data")

In [ ]:
# For analytics_keepers
print("MISSING DATA ANALYSIS - ANALYTICS_KEEPERS (Current):")
print("=" * 100)

null_counts_keepers = conn.execute("""
    SELECT 
        COUNT(*) as total_records,
        SUM(CASE WHEN clean_sheets IS NULL THEN 1 ELSE 0 END) as null_clean_sheets,
        SUM(CASE WHEN goals_against IS NULL THEN 1 ELSE 0 END) as null_goals_against,
        SUM(CASE WHEN saves IS NULL THEN 1 ELSE 0 END) as null_saves,
        SUM(CASE WHEN save_percentage IS NULL THEN 1 ELSE 0 END) as null_save_percentage,
        SUM(CASE WHEN minutes_played IS NULL THEN 1 ELSE 0 END) as null_minutes,
        SUM(CASE WHEN matches_played IS NULL THEN 1 ELSE 0 END) as null_matches
    FROM analytics_keepers
    WHERE is_current = true
""").fetchdf()

total_keepers = null_counts_keepers['total_records'].iloc[0]
if total_keepers > 0:
    null_pct_clean_sheets = (null_counts_keepers['null_clean_sheets'].iloc[0] / total_keepers) * 100
    null_pct_goals_against = (null_counts_keepers['null_goals_against'].iloc[0] / total_keepers) * 100
    null_pct_saves = (null_counts_keepers['null_saves'].iloc[0] / total_keepers) * 100
    null_pct_save_pct = (null_counts_keepers['null_save_percentage'].iloc[0] / total_keepers) * 100
    null_pct_minutes_k = (null_counts_keepers['null_minutes'].iloc[0] / total_keepers) * 100
    null_pct_matches_k = (null_counts_keepers['null_matches'].iloc[0] / total_keepers) * 100
    
    missing_summary_keepers = pd.DataFrame({
        'Column': ['clean_sheets', 'goals_against', 'saves', 'save_percentage', 'minutes_played', 'matches_played'],
        'Null Count': [
            null_counts_keepers['null_clean_sheets'].iloc[0],
            null_counts_keepers['null_goals_against'].iloc[0],
            null_counts_keepers['null_saves'].iloc[0],
            null_counts_keepers['null_save_percentage'].iloc[0],
            null_counts_keepers['null_minutes'].iloc[0],
            null_counts_keepers['null_matches'].iloc[0]
        ],
        'Missing %': [null_pct_clean_sheets, null_pct_goals_against, null_pct_saves, 
                      null_pct_save_pct, null_pct_minutes_k, null_pct_matches_k]
    })
    
    print(missing_summary_keepers.to_string(index=False))
    
    problematic_keepers = missing_summary_keepers[missing_summary_keepers['Missing %'] > 5]
    if len(problematic_keepers) > 0:
        print(f"\n⚠️  WARNING: {len(problematic_keepers)} columns have >5% missing data:")
        print(problematic_keepers[['Column', 'Missing %']].to_string(index=False))
    else:
        print("\n✅ All columns have <5% missing data")
else:
    print("No keeper records found.")

In [ ]:
# For analytics_squads
print("MISSING DATA ANALYSIS - ANALYTICS_SQUADS (Current):")
print("=" * 100)

null_counts_squads = conn.execute("""
    SELECT 
        COUNT(*) as total_records,
        SUM(CASE WHEN goals IS NULL THEN 1 ELSE 0 END) as null_goals,
        SUM(CASE WHEN goals_against IS NULL THEN 1 ELSE 0 END) as null_goals_against,
        SUM(CASE WHEN wins IS NULL THEN 1 ELSE 0 END) as null_wins,
        SUM(CASE WHEN draws IS NULL THEN 1 ELSE 0 END) as null_draws,
        SUM(CASE WHEN losses IS NULL THEN 1 ELSE 0 END) as null_losses,
        SUM(CASE WHEN matches_played IS NULL THEN 1 ELSE 0 END) as null_matches
    FROM analytics_squads
    WHERE is_current = true
""").fetchdf()

total_squads = null_counts_squads['total_records'].iloc[0]
if total_squads > 0:
    null_pct_goals_sq = (null_counts_squads['null_goals'].iloc[0] / total_squads) * 100
    null_pct_goals_against_sq = (null_counts_squads['null_goals_against'].iloc[0] / total_squads) * 100
    null_pct_wins = (null_counts_squads['null_wins'].iloc[0] / total_squads) * 100
    null_pct_draws = (null_counts_squads['null_draws'].iloc[0] / total_squads) * 100
    null_pct_losses = (null_counts_squads['null_losses'].iloc[0] / total_squads) * 100
    null_pct_matches_sq = (null_counts_squads['null_matches'].iloc[0] / total_squads) * 100
    
    missing_summary_squads = pd.DataFrame({
        'Column': ['goals', 'goals_against', 'wins', 'draws', 'losses', 'matches_played'],
        'Null Count': [
            null_counts_squads['null_goals'].iloc[0],
            null_counts_squads['null_goals_against'].iloc[0],
            null_counts_squads['null_wins'].iloc[0],
            null_counts_squads['null_draws'].iloc[0],
            null_counts_squads['null_losses'].iloc[0],
            null_counts_squads['null_matches'].iloc[0]
        ],
        'Missing %': [null_pct_goals_sq, null_pct_goals_against_sq, null_pct_wins, 
                      null_pct_draws, null_pct_losses, null_pct_matches_sq]
    })
    
    print(missing_summary_squads.to_string(index=False))
    
    problematic_squads = missing_summary_squads[missing_summary_squads['Missing %'] > 5]
    if len(problematic_squads) > 0:
        print(f"\n⚠️  WARNING: {len(problematic_squads)} columns have >5% missing data:")
        print(problematic_squads[['Column', 'Missing %']].to_string(index=False))
    else:
        print("\n✅ All columns have <5% missing data")
else:
    print("No squad records found.")

## 3.4 Data Quality Checks (Current)

In [ ]:
# Check 1: Players with minutes > matches_played * 90
print("DATA QUALITY CHECK 1: Players with excessive minutes")
print("=" * 100)

invalid_minutes = conn.execute("""
    SELECT player_name, squad, minutes_played, matches_played,
           minutes_played - (matches_played * 90) as minutes_over
    FROM analytics_players
    WHERE is_current = true
      AND minutes_played > matches_played * 90 + 15
    ORDER BY minutes_over DESC
""").fetchdf()

if len(invalid_minutes) > 0:
    print(f"⚠️  Found {len(invalid_minutes)} players with minutes > matches*90 + 15 (allowing for added time)")
    print(invalid_minutes.head(10).to_string(index=False))
    if len(invalid_minutes) > 10:
        print(f"\n... and {len(invalid_minutes) - 10} more")
else:
    print("✅ No players with excessive minutes found")

In [ ]:
# Check 2: Negative values in count columns
print("\nDATA QUALITY CHECK 2: Negative values in stat columns")
print("=" * 100)

negative_values = conn.execute("""
    SELECT player_name, squad,
           goals, assists, shots, passes_completed
    FROM analytics_players
    WHERE is_current = true
      AND (goals < 0 OR assists < 0 OR shots < 0 OR passes_completed < 0)
""").fetchdf()

if len(negative_values) > 0:
    print(f"⚠️  Found {len(negative_values)} players with negative stat values")
    print(negative_values.to_string(index=False))
else:
    print("✅ No negative values found in stat columns")

In [ ]:
# Check 3: Squads with gameweek != matches_played (should be close)
print("\nDATA QUALITY CHECK 3: Squads with gameweek/matches mismatch")
print("=" * 100)

gw_matches_mismatch = conn.execute("""
    SELECT squad_name, gameweek, matches_played,
           ABS(gameweek - matches_played) as difference
    FROM analytics_squads
    WHERE is_current = true
      AND ABS(gameweek - matches_played) > 2
    ORDER BY difference DESC
""").fetchdf()

if len(gw_matches_mismatch) > 0:
    print(f"⚠️  Found {len(gw_matches_mismatch)} squads with gameweek/matches difference > 2")
    print(gw_matches_mismatch.to_string(index=False))
else:
    print("✅ All squads have consistent gameweek/matches_played values (within 2)")

# Section 4: HISTORICAL DATA INSPECTION

Examining the full temporal dataset to understand coverage and trends.

## 4.1 Temporal Coverage

In [ ]:
# Temporal coverage - Multi-season analysis
print("TEMPORAL COVERAGE ANALYSIS")
print("=" * 100)

# Get season-level summary
season_summary = conn.execute("""
    SELECT 
        season,
        COUNT(DISTINCT gameweek) as num_gameweeks,
        MIN(gameweek) as min_gw,
        MAX(gameweek) as max_gw,
        COUNT(*) as total_records,
        SUM(CASE WHEN is_current = true THEN 1 ELSE 0 END) as current_records
    FROM analytics_squads
    GROUP BY season
    ORDER BY season
""").fetchdf()

print("SEASON-BY-SEASON BREAKDOWN:")
print(season_summary.to_string(index=False))

# Overall summary
total_seasons = len(season_summary)
historical_seasons = len(season_summary[season_summary['season'] != '2025-2026'])
current_season_gws = season_summary[season_summary['season'] == '2025-2026']['num_gameweeks'].iloc[0]

print(f"\n{'=' * 100}")
print("OVERALL TEMPORAL COVERAGE SUMMARY:")
print(f"{'=' * 100}")
print(f"Total Seasons: {total_seasons}")
print(f"Historical Seasons (complete): {historical_seasons} (2010-2011 to 2024-2025)")
print(f"Current Season (in-progress): 2025-2026 (GW 3-7, {int(current_season_gws)} gameweeks)")
print(f"\nHistorical Data Structure:")
print(f"  - End-of-season snapshots only (GW 38)")
print(f"  - {historical_seasons * 20} total squad records")
print(f"  - Provides final standings/outcomes for 15 years")
print(f"\nCurrent Season Data Structure:")
print(f"  - Week-by-week progression (GW 3-7)")
print(f"  - {int(current_season_gws) * 20} total squad records")
print(f"  - GW 7 = current state (is_current = true)")

print(f"\n{'=' * 100}")
print("DATA CHARACTERISTICS:")
print(f"{'=' * 100}")
print("✅ AVAILABLE:")
print("   - 15 years of final season outcomes (GW 38)")
print("   - 5 gameweeks of current season progression (GW 3-7)")
print("   - Historical benchmarks for final standings")
print("\n❌ NOT AVAILABLE:")
print("   - Historical week-by-week progression (GW 1-37 for past seasons)")
print("   - Full season time-series for historical years")
print("\n💡 MODELING IMPLICATIONS:")
print("   ✅ Can predict: GW 7 form → final GW 38 outcome")
print("   ✅ Can compare: Current teams vs historical final positions")
print("   ❌ Cannot model: Historical week-to-week progression patterns")

## 4.2 Records Per Gameweek

In [ ]:
# Records per gameweek - with season context
print("RECORDS PER GAMEWEEK BY SEASON:")
print("=" * 100)

# Get records by season and gameweek
records_by_season_gw = conn.execute("""
    SELECT 
        season,
        gameweek,
        COUNT(*) as total_records,
        COUNT(DISTINCT squad_name) as unique_squads
    FROM analytics_squads
    GROUP BY season, gameweek
    ORDER BY season, gameweek
""").fetchdf()

# Show historical seasons (GW 38 only)
print("\nHISTORICAL SEASONS (End-of-Season Snapshots):")
print("-" * 100)
historical_records = records_by_season_gw[records_by_season_gw['season'] != '2025-2026']
print(historical_records.to_string(index=False))

print(f"\nAll {len(historical_records)} historical seasons have GW 38 only (final standings)")
print(f"Total historical records: {historical_records['total_records'].sum()}")

# Show current season (GW 3-7)
print("\n" + "=" * 100)
print("CURRENT SEASON (Week-by-Week Progression):")
print("-" * 100)
current_records = records_by_season_gw[records_by_season_gw['season'] == '2025-2026']
print(current_records.to_string(index=False))

# Visualize current season progression
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Current season gameweek progression
ax1.plot(current_records['gameweek'], current_records['total_records'], 
         marker='o', linewidth=3, markersize=8, color='#2ca02c')
ax1.axhline(y=20, color='red', linestyle='--', alpha=0.5, label='Expected (20 squads)')
ax1.set_xlabel('Gameweek', fontsize=12)
ax1.set_ylabel('Number of Records', fontsize=12)
ax1.set_title('Current Season (2025-2026) - Records per Gameweek', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.set_xticks(current_records['gameweek'])

# Plot 2: Historical vs Current comparison
ax2.bar(['Historical\n(15 seasons × GW38)', 'Current\n(GW 3-7)'], 
        [historical_records['total_records'].sum(), current_records['total_records'].sum()],
        color=['#1f77b4', '#2ca02c'])
ax2.set_ylabel('Total Records', fontsize=12)
ax2.set_title('Data Distribution: Historical vs Current', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)
for i, v in enumerate([historical_records['total_records'].sum(), current_records['total_records'].sum()]):
    ax2.text(i, v + 5, str(int(v)), ha='center', va='bottom', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.savefig(output_dir / "historical_records_per_gw.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Chart saved to: {output_dir / 'historical_records_per_gw.png'}")

## 4.3 Missing Data Trends

In [ ]:
# Missing data trends - by season
print("MISSING DATA ANALYSIS BY SEASON:")
print("=" * 100)

missing_by_season = conn.execute("""
    SELECT 
        season,
        COUNT(*) as total_players,
        SUM(CASE WHEN goals IS NULL THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as pct_null_goals,
        SUM(CASE WHEN assists IS NULL THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as pct_null_assists,
        SUM(CASE WHEN minutes_played IS NULL THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as pct_null_minutes
    FROM analytics_players
    GROUP BY season
    ORDER BY season
""").fetchdf()

print("Missing Data % by Season:")
print(missing_by_season.to_string(index=False))

# Visualize
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Plot 1: Goals
axes[0].bar(range(len(missing_by_season)), missing_by_season['pct_null_goals'], color='#e377c2')
axes[0].axhline(y=5, color='red', linestyle='--', alpha=0.5, label='5% threshold')
axes[0].set_ylabel('Missing %', fontsize=12)
axes[0].set_title('Missing Goals Data % by Season', fontsize=14, fontweight='bold')
axes[0].set_xticks(range(len(missing_by_season)))
axes[0].set_xticklabels(missing_by_season['season'], rotation=45, ha='right')
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].legend()

# Plot 2: Assists
axes[1].bar(range(len(missing_by_season)), missing_by_season['pct_null_assists'], color='#ff7f0e')
axes[1].axhline(y=5, color='red', linestyle='--', alpha=0.5, label='5% threshold')
axes[1].set_ylabel('Missing %', fontsize=12)
axes[1].set_title('Missing Assists Data % by Season', fontsize=14, fontweight='bold')
axes[1].set_xticks(range(len(missing_by_season)))
axes[1].set_xticklabels(missing_by_season['season'], rotation=45, ha='right')
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].legend()

# Plot 3: Minutes
axes[2].bar(range(len(missing_by_season)), missing_by_season['pct_null_minutes'], color='#2ca02c')
axes[2].axhline(y=5, color='red', linestyle='--', alpha=0.5, label='5% threshold')
axes[2].set_xlabel('Season', fontsize=12)
axes[2].set_ylabel('Missing %', fontsize=12)
axes[2].set_title('Missing Minutes Data % by Season', fontsize=14, fontweight='bold')
axes[2].set_xticks(range(len(missing_by_season)))
axes[2].set_xticklabels(missing_by_season['season'], rotation=45, ha='right')
axes[2].grid(True, alpha=0.3, axis='y')
axes[2].legend()

plt.tight_layout()
plt.savefig(output_dir / "missing_data_trends.png", dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Chart saved to: {output_dir / 'missing_data_trends.png'}")

# Summary
max_missing = missing_by_season[['pct_null_goals', 'pct_null_assists', 'pct_null_minutes']].max().max()
print(f"\n📊 Maximum missing data across all seasons: {max_missing:.2f}%")
if max_missing < 5:
    print("✅ Excellent - All seasons have <5% missing data")
elif max_missing < 10:
    print("⚠️  Good - Some seasons have 5-10% missing data")
else:
    print("❌ Warning - Some seasons have >10% missing data")

## 4.4 SCD Type 2 Validation

In [ ]:
# Check SCD Type 2 integrity
print("SCD TYPE 2 VALIDATION:")
print("=" * 100)

scd_validation = conn.execute("""
    SELECT 
        COUNT(*) as total_records,
        SUM(CASE WHEN valid_from IS NULL THEN 1 ELSE 0 END) as null_valid_from,
        SUM(CASE WHEN valid_to IS NULL AND is_current = false THEN 1 ELSE 0 END) as historical_without_valid_to,
        SUM(CASE WHEN valid_to IS NOT NULL AND is_current = true THEN 1 ELSE 0 END) as current_with_valid_to,
        SUM(CASE WHEN valid_to < valid_from THEN 1 ELSE 0 END) as invalid_date_range
    FROM analytics_players
""").fetchdf()

print("SCD Type 2 Validation - analytics_players:")
print(scd_validation.to_string(index=False))

issues_found = False
if scd_validation['null_valid_from'].iloc[0] > 0:
    print(f"\n⚠️  WARNING: {scd_validation['null_valid_from'].iloc[0]} records with NULL valid_from")
    issues_found = True
    
if scd_validation['historical_without_valid_to'].iloc[0] > 0:
    print(f"⚠️  WARNING: {scd_validation['historical_without_valid_to'].iloc[0]} historical records without valid_to")
    issues_found = True
    
if scd_validation['current_with_valid_to'].iloc[0] > 0:
    print(f"⚠️  WARNING: {scd_validation['current_with_valid_to'].iloc[0]} current records with valid_to set")
    issues_found = True
    
if scd_validation['invalid_date_range'].iloc[0] > 0:
    print(f"⚠️  WARNING: {scd_validation['invalid_date_range'].iloc[0]} records with valid_to < valid_from")
    issues_found = True

if not issues_found:
    print("\n✅ SCD Type 2 tracking is working correctly")

# Find players with multiple current records (should be 0)
print("\n" + "=" * 100)
print("CHECKING FOR DUPLICATE CURRENT RECORDS:")
print("=" * 100)

duplicate_current = conn.execute("""
    SELECT player_id, player_name, squad, COUNT(*) as num_current_records
    FROM analytics_players
    WHERE is_current = true
    GROUP BY player_id, player_name, squad
    HAVING COUNT(*) > 1
""").fetchdf()

if len(duplicate_current) > 0:
    print(f"⚠️  WARNING: Found {len(duplicate_current)} players with duplicate current records!")
    print(duplicate_current.to_string(index=False))
else:
    print("✅ No duplicate current records found")

## 4.5 Gameweek Consistency Check

In [ ]:
# Verify all players on same team have same gameweek per snapshot
print("GAMEWEEK CONSISTENCY CHECK:")
print("=" * 100)
print("Verifying that all players on each squad have consistent gameweeks per snapshot...\n")

gw_consistency = conn.execute("""
    WITH squad_gws AS (
        SELECT 
            squad,
            valid_from,
            COUNT(DISTINCT gameweek) as gw_variations
        FROM analytics_players
        GROUP BY squad, valid_from
        HAVING COUNT(DISTINCT gameweek) > 1
    )
    SELECT * FROM squad_gws
""").fetchdf()

if len(gw_consistency) > 0:
    print(f"⚠️  WARNING: Found {len(gw_consistency)} squad snapshots with inconsistent gameweeks")
    print(gw_consistency.to_string(index=False))
else:
    print("✅ All players on each squad have consistent gameweeks per snapshot")

# Section 5: Summary Report Generation

## 5.1 Create Comprehensive Text Report

In [ ]:
from pathlib import Path

# Create output directory (already exists, but ensuring)
output_dir = Path("../../outputs/01_data_inspection")
output_dir.mkdir(parents=True, exist_ok=True)

report_path = output_dir / "data_inspection_report.txt"

with open(report_path, 'w') as f:
    f.write("=" * 80 + "\n")
    f.write("PREMIER LEAGUE ANALYTICS DATABASE - INSPECTION REPORT\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("=" * 80 + "\n\n")
    
    # SECTION 1: TABLE OVERVIEW
    f.write("SECTION 1: TABLE OVERVIEW\n")
    f.write("-" * 80 + "\n")
    f.write(f"analytics_players:\n")
    f.write(f"  - Total records: {total_rows_players:,}\n")
    f.write(f"  - Current records: {current_rows_players:,}\n")
    f.write(f"  - Columns: {len(table_info_players)}\n\n")
    
    f.write(f"analytics_keepers:\n")
    f.write(f"  - Total records: {total_rows_keepers:,}\n")
    f.write(f"  - Current records: {current_rows_keepers:,}\n")
    f.write(f"  - Columns: {len(table_info_keepers)}\n\n")
    
    f.write(f"analytics_squads:\n")
    f.write(f"  - Total records: {total_rows_squads:,}\n")
    f.write(f"  - Current records: {current_rows_squads:,}\n")
    f.write(f"  - Columns: {len(table_info_squads)}\n\n")
    
    f.write(f"analytics_fixtures:\n")
    f.write(f"  - Total records: {total_rows_fixtures:,}\n")
    f.write(f"  - Columns: {len(table_info_fixtures)}\n\n")
    
    # SECTION 2: CURRENT DATA SUMMARY
    f.write("\nSECTION 2: CURRENT DATA SUMMARY (is_current=true)\n")
    f.write("-" * 80 + "\n")
    f.write(f"Current Squads: {squad_count}\n")
    f.write(f"Current Players (outfield): {current_rows_players}\n")
    f.write(f"Current Keepers: {keeper_count}\n\n")
    
    f.write("Gameweek Distribution (Current Season):\n")
    for _, row in gw_distribution_squads.iterrows():
        f.write(f"  GW{row['gameweek']}: {row['num_squads']} squads\n")
    f.write("\n")
    
    # SECTION 3: MULTI-SEASON DATA STRUCTURE
    f.write("\nSECTION 3: MULTI-SEASON DATA STRUCTURE\n")
    f.write("-" * 80 + "\n")
    f.write(f"Total Seasons: {total_seasons}\n")
    f.write(f"Historical Seasons: {historical_seasons} (2010-2011 to 2024-2025)\n")
    f.write(f"Current Season: 2025-2026 (GW 3-7, {int(current_season_gws)} gameweeks)\n\n")
    
    f.write("Historical Data Characteristics:\n")
    f.write(f"  - End-of-season snapshots only (GW 38)\n")
    f.write(f"  - {historical_seasons * 20} total squad records\n")
    f.write(f"  - Provides final standings/outcomes for {historical_seasons} years\n\n")
    
    f.write("Current Season Data Characteristics:\n")
    f.write(f"  - Week-by-week progression available (GW 3-7)\n")
    f.write(f"  - {int(current_season_gws) * 20} total squad records\n")
    f.write(f"  - GW 7 = current state (is_current = true)\n")
    
    # SECTION 4: DATA QUALITY ISSUES
    f.write("\nSECTION 4: DATA QUALITY ISSUES FOUND\n")
    f.write("-" * 80 + "\n")
    
    issue_count = 0
    
    if len(invalid_minutes) > 0:
        f.write(f"⚠️  Issue 1: {len(invalid_minutes)} players with minutes > matches*90\n")
        issue_count += 1
    
    if len(negative_values) > 0:
        f.write(f"⚠️  Issue 2: {len(negative_values)} players with negative stat values\n")
        issue_count += 1
    
    if len(gw_matches_mismatch) > 0:
        f.write(f"⚠️  Issue 3: {len(gw_matches_mismatch)} squads with gameweek/matches mismatch\n")
        issue_count += 1
    
    if len(duplicate_current) > 0:
        f.write(f"⚠️  Issue 4: {len(duplicate_current)} players with duplicate current records\n")
        issue_count += 1
    
    if len(gw_consistency) > 0:
        f.write(f"⚠️  Issue 5: {len(gw_consistency)} squads with inconsistent gameweeks\n")
        issue_count += 1
    
    if issue_count == 0:
        f.write("✅ No critical data quality issues found!\n")
    else:
        f.write(f"\nTotal Issues Found: {issue_count}\n")
    
    # SECTION 5: MISSING DATA SUMMARY
    f.write("\nSECTION 5: MISSING DATA SUMMARY (Current Data)\n")
    f.write("-" * 80 + "\n")
    
    f.write("analytics_players:\n")
    f.write(f"  Goals: {null_pct_goals:.1f}% missing\n")
    f.write(f"  Assists: {null_pct_assists:.1f}% missing\n")
    f.write(f"  Minutes: {null_pct_minutes:.1f}% missing\n\n")
    
    max_missing_pct = max(null_pct_goals, null_pct_assists, null_pct_minutes)
    if max_missing_pct > 5:
        f.write("⚠️  WARNING: Some columns have >5% missing data\n")
    else:
        f.write("✅ All key columns have <5% missing data\n")
    
    # SECTION 6: READINESS FOR ML
    f.write("\nSECTION 6: ML READINESS ASSESSMENT\n")
    f.write("-" * 80 + "\n")
    
    # Calculate overall score - deduct 10 for no historical week-by-week progression
    ml_ready_score = 100
    if issue_count > 0:
        ml_ready_score -= (issue_count * 10)
    if max_missing_pct > 5:
        ml_ready_score -= 20
    # Deduct 10 for limited historical progression (only GW 38, not full season)
    ml_ready_score -= 10
    
    f.write(f"Overall Data Quality Score: {ml_ready_score}/100\n\n")
    
    f.write("Score Breakdown:\n")
    f.write(f"  Base Score: 100\n")
    f.write(f"  - Data Quality Issues: -{issue_count * 10} ({issue_count} issues found)\n")
    f.write(f"  - Missing Data: -{20 if max_missing_pct > 5 else 0} ({'high' if max_missing_pct > 5 else 'minimal'} missing data)\n")
    f.write(f"  - Limited Historical Progression: -10 (only GW 38, not full season)\n\n")
    
    if ml_ready_score >= 80:
        f.write("✅ GREEN LIGHT: Data is ready for ML modeling\n")
        readiness_status = "GREEN"
    elif ml_ready_score >= 60:
        f.write("⚠️  YELLOW LIGHT: Data is usable but has minor limitations\n")
        readiness_status = "YELLOW"
    else:
        f.write("🛑 RED LIGHT: Data has significant issues that should be addressed\n")
        readiness_status = "RED"
    
    # SECTION 7: RECOMMENDATIONS
    f.write("\nSECTION 7: RECOMMENDATIONS\n")
    f.write("-" * 80 + "\n")
    
    f.write("1. DATA STRUCTURE UNDERSTANDING:\n")
    f.write(f"   ✅ {historical_seasons} years of final season outcomes (GW 38)\n")
    f.write(f"   ✅ {int(current_season_gws)} gameweeks of current season progression (GW 3-7)\n")
    f.write(f"   ❌ No historical week-by-week progression (GW 1-37 for past seasons)\n\n")
    
    f.write("2. RECOMMENDED MODELING APPROACHES:\n")
    f.write("   A. Final Standings Prediction:\n")
    f.write("      - Use GW 7 form to predict final GW 38 outcome\n")
    f.write(f"      - Train on {historical_seasons} years of historical benchmarks\n")
    f.write("      - Compare current teams vs historical final positions\n\n")
    f.write("   B. Next Match Prediction:\n")
    f.write("      - Use current form (GW 7) to predict upcoming matches\n")
    f.write("      - Historical GW 38 provides team quality indicators\n\n")
    
    f.write("3. TRAIN-TEST SPLIT STRATEGY:\n")
    f.write("   ⚠️  IMPORTANT: Account for multi-season structure\n")
    f.write("   ✅ Option A: Train on 2010-2023, validate on 2024, test on 2025\n")
    f.write("   ✅ Option B: Leave-one-season-out cross-validation\n")
    f.write("   ❌ DO NOT: Random shuffle across seasons\n\n")
    
    f.write("4. NEXT STEPS:\n")
    f.write("   - Proceed to Part 1B: Goals Analysis\n")
    f.write("   - Continue exploratory analysis through Parts 1C-1H\n")
    f.write("   - Begin feature engineering focused on GW 7 → GW 38 prediction\n")
    if issue_count > 0:
        f.write("   - Address critical data quality issues found\n")
    f.write("\n")

print(f"✅ Detailed report saved to: {report_path}")

# Store key variables for later use
globals()['ml_ready_score'] = ml_ready_score
globals()['readiness_status'] = readiness_status
globals()['issue_count'] = issue_count
globals()['max_missing_pct'] = max_missing_pct

## 5.2 Create Visual Summary Dashboard

In [ ]:
# Create comprehensive visual summary
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.4, wspace=0.3)

# Subplot 1: Record counts by table (current)
ax1 = fig.add_subplot(gs[0, 0])
table_counts = pd.DataFrame({
    'Table': ['Squads', 'Players', 'Keepers'],
    'Count': [squad_count, current_rows_players, keeper_count]
})
ax1.bar(table_counts['Table'], table_counts['Count'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax1.set_title('Current Record Counts by Table', fontweight='bold')
ax1.set_ylabel('Number of Records')
for i, v in enumerate(table_counts['Count']):
    ax1.text(i, v + 5, str(v), ha='center', va='bottom', fontweight='bold')

# Subplot 2: Gameweek distribution (current)
ax2 = fig.add_subplot(gs[0, 1])
ax2.bar(gw_distribution_squads['gameweek'], gw_distribution_squads['num_squads'], color='#d62728')
ax2.set_title('Current Gameweek Distribution', fontweight='bold')
ax2.set_xlabel('Gameweek')
ax2.set_ylabel('Number of Squads')
ax2.set_xticks(gw_distribution_squads['gameweek'])

# Subplot 3: Missing data % (current key columns)
ax3 = fig.add_subplot(gs[0, 2])
missing_data_summary = pd.DataFrame({
    'Column': ['Goals', 'Assists', 'Minutes'],
    'Missing %': [null_pct_goals, null_pct_assists, null_pct_minutes]
})
colors = ['green' if x < 5 else 'orange' if x < 10 else 'red' for x in missing_data_summary['Missing %']]
ax3.barh(missing_data_summary['Column'], missing_data_summary['Missing %'], color=colors)
ax3.set_title('Missing Data % (Current)', fontweight='bold')
ax3.set_xlabel('Missing %')
ax3.axvline(x=5, color='red', linestyle='--', alpha=0.5, label='5% threshold')
ax3.legend()

# Subplot 4: Multi-season data distribution
ax4 = fig.add_subplot(gs[1, :])
historical_total = historical_records['total_records'].sum()
current_total = current_records['total_records'].sum()
season_types = ['Historical\n(15 seasons × GW38)', 'Current\n(2025-2026, GW 3-7)']
season_counts = [historical_total, current_total]
bars = ax4.bar(season_types, season_counts, color=['#1f77b4', '#2ca02c'], width=0.6)
ax4.set_ylabel('Total Squad Records', fontsize=12)
ax4.set_title('Multi-Season Data Distribution', fontsize=14, fontweight='bold')
ax4.grid(axis='y', alpha=0.3)
for bar, count in zip(bars, season_counts):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 5,
             f'{int(count)} records',
             ha='center', va='bottom', fontweight='bold', fontsize=11)

# Subplot 5: Missing data by season (first 3 columns)
ax5 = fig.add_subplot(gs[2, 0])
season_indices = range(len(missing_by_season))
ax5.plot(season_indices, missing_by_season['pct_null_goals'], marker='o', color='#e377c2', linewidth=2, label='Goals')
ax5.axhline(y=5, color='red', linestyle='--', alpha=0.5)
ax5.set_title('Missing Data % Across Seasons', fontweight='bold', fontsize=10)
ax5.set_xlabel('Season Index')
ax5.set_ylabel('Missing %')
ax5.grid(True, alpha=0.3)
ax5.legend()

# Subplot 6: Player counts by position (current)
ax6 = fig.add_subplot(gs[2, 1])
top_positions = position_counts.head(6)  # Show top 6 positions for clarity
ax6.bar(top_positions['position'], top_positions['count'], color='#17becf')
ax6.set_title('Players by Position (Current)', fontweight='bold', fontsize=10)
ax6.set_xlabel('Position')
ax6.set_ylabel('Number of Players')
ax6.tick_params(axis='x', rotation=45)

# Subplot 7: Fixtures completion status
ax7 = fig.add_subplot(gs[2, 2])
fixtures_labels = ['Completed' if x else 'Pending' for x in fixtures_breakdown['is_completed']]
ax7.pie(fixtures_breakdown['count'], labels=fixtures_labels, autopct='%1.1f%%', colors=['#2ca02c', '#ff7f0e'])
ax7.set_title('Fixtures Status', fontweight='bold')

plt.suptitle('Premier League Analytics Database - Visual Summary', fontsize=18, fontweight='bold', y=0.995)

# Save figure
visual_summary_path = output_dir / "visual_summary.png"
plt.savefig(visual_summary_path, dpi=300, bbox_inches='tight')
print(f"✅ Visual summary saved to: {visual_summary_path}")
plt.show()

## 5.3 Save Additional Output Files

In [ ]:
# Save table schemas
schema_path = output_dir / "table_schemas.txt"
with open(schema_path, 'w') as f:
    for table in ['analytics_players', 'analytics_keepers', 'analytics_squads', 'analytics_fixtures']:
        f.write(f"\n{'=' * 80}\n")
        f.write(f"TABLE: {table}\n")
        f.write(f"{'=' * 80}\n\n")
        
        schema = conn.execute(f"PRAGMA table_info({table})").fetchdf()
        f.write(schema.to_string(index=False))
        f.write("\n\n")

print(f"✅ Table schemas saved to: {schema_path}")

# Save data quality issues detail
issues_path = output_dir / "data_quality_issues.txt"
with open(issues_path, 'w') as f:
    f.write("DATA QUALITY ISSUES - DETAILED LISTING\n")
    f.write("=" * 80 + "\n\n")
    
    if len(invalid_minutes) > 0:
        f.write("ISSUE 1: Players with minutes > matches*90\n")
        f.write("-" * 80 + "\n")
        f.write(invalid_minutes.to_string(index=False))
        f.write("\n\n")
    
    if len(negative_values) > 0:
        f.write("ISSUE 2: Players with negative stat values\n")
        f.write("-" * 80 + "\n")
        f.write(negative_values.to_string(index=False))
        f.write("\n\n")
    
    if len(gw_matches_mismatch) > 0:
        f.write("ISSUE 3: Squads with gameweek/matches mismatch\n")
        f.write("-" * 80 + "\n")
        f.write(gw_matches_mismatch.to_string(index=False))
        f.write("\n\n")
    
    if len(duplicate_current) > 0:
        f.write("ISSUE 4: Players with duplicate current records\n")
        f.write("-" * 80 + "\n")
        f.write(duplicate_current.to_string(index=False))
        f.write("\n\n")
    
    if len(gw_consistency) > 0:
        f.write("ISSUE 5: Squads with inconsistent gameweeks\n")
        f.write("-" * 80 + "\n")
        f.write(gw_consistency.to_string(index=False))
        f.write("\n\n")
    
    if issue_count == 0:
        f.write("✅ No data quality issues found!\n")

print(f"✅ Data quality issues saved to: {issues_path}")

# Save sample data
sample_path = output_dir / "sample_data.csv"
sample_combined = pd.concat([
    sample_current_players.head(3),
    sample_historical_players.head(3)
], ignore_index=True)
sample_combined.to_csv(sample_path, index=False)
print(f"✅ Sample data saved to: {sample_path}")

# Section 6: Key Findings and Conclusions

## Summary of Key Findings

### 1. Data Availability ✅

**Multi-Season Structure:**
- **16 Total Seasons:** 15 historical + 1 current (2010-2011 through 2025-2026)
- **Historical Seasons (2010-2025):** 15 seasons with GW 38 snapshots only (300 squad records)
  - Provides end-of-season outcomes and final standings
  - 15 years of benchmarks for final positions, goals, points
- **Current Season (2025-2026):** Week-by-week progression available
  - GW 3-7 tracked (100 squad records across 5 gameweeks)
  - GW 7 = current state (is_current = true)
  - 20 squads, 404 players, 28 keepers

**Fixture Data:**
- Total fixtures: 380 matches
- Completed: 60 matches (15.8%)
- Pending: 320 matches (84.2%)

### 2. Data Quality Score: 90/100 🟢

**Rating:** GREEN LIGHT - Data is ready for ML modeling

**Score Breakdown:**
- Base Score: 100
- Deductions:
  - Data quality issues: -0 (none found)
  - Missing data: -0 (0% missing in current records)
  - Limited historical progression: -10 (only GW 38, not full season history)

**Justification:** Excellent data quality with 0% missing data and 0 logical consistency errors. Only limitation is historical data shows end-of-season snapshots rather than week-by-week progression.

### 3. Critical Issues 🚨

**None found!** ✅

All validation checks passed:
- ✅ No players with excessive minutes
- ✅ No negative stat values
- ✅ No gameweek/matches mismatches
- ✅ No duplicate current records
- ✅ Gameweek consistency maintained across all squads

### 4. Green Lights ✅

- **SCD Type 2 tracking:** Perfect - all records have valid_from, historical records properly closed with valid_to
- **Gameweek consistency:** Perfect - all players on same squad share same gameweek per snapshot
- **Missing data:** Excellent - 0% missing in all key columns (goals, assists, minutes, matches)
- **Logical consistency:** Perfect - no data integrity violations found
- **Multi-season tracking:** Well-structured - 16 seasons properly tracked with season field

### 5. Ready for ML Modeling?

**Decision:** YES - Proceed with snapshot-based modeling ✅

**Data Strengths:**
- 15 years of historical final outcomes (GW 38) provide strong benchmarks
- Current season progression (GW 3-7) captures recent form
- Perfect data quality (0% missing, 0 errors)
- 60 completed matches for training labels

**Modeling Approach Recommendation:**
- **Primary Model:** Predict final GW 38 outcome from current GW 7 form
  - Train on 15 years of historical benchmarks
  - Use current season's GW 3-7 for feature engineering
- **Alternative Model:** Next match prediction using current form + historical quality indicators

### 6. Next Steps

1. ✅ **Part 1A Complete** - Multi-season data structure understood
2. ⏭️  **Part 1B** - Goals Analysis (descriptive statistics across seasons)
3. ⏭️  **Parts 1C-1H** - Continue exploratory analysis with season awareness
4. 📊 **Feature Engineering Focus:** 
   - Current form metrics (GW 3-7 trends)
   - Historical quality indicators (past GW 38 outcomes)
   - Team comparison features (current vs historical averages)

### 7. Notes for Future Reference

**Data Structure Insights:**
- 15 historical seasons provide end-of-season snapshots (GW 38 only)
- Current season (2025-2026) has week-by-week tracking (GW 3-7)
- Cannot build historical time-series models (no GW 1-37 for past seasons)
- **Modeling sweet spot:** Current form (GW 7) → Final outcome (GW 38)

**Technical Considerations:**
- SCD Type 2 tracking with is_current flag working perfectly
- Multi-season structure requires season-aware train/test splits
- Must use chronological splits (NOT random shuffle)
- Recommended: Train on 2010-2023, validate on 2024, test on 2025
- Account for squad quality differences across seasons

**Data Quality:**
- **Perfect completeness:** 0% missing data in current records
- **Perfect consistency:** All logical checks passed
- **High confidence:** GREEN LIGHT to proceed with ML development

# Section 7: Cleanup and Final Checks

In [ ]:
# Close database connection
conn.close()
print("\n" + "=" * 80)
print("✅ PART 1A: DATA INSPECTION COMPLETE")
print("=" * 80)
print(f"\n📂 All outputs saved to: {output_dir}")
print(f"\n📊 Files created:")
print(f"   - data_inspection_report.txt")
print(f"   - visual_summary.png")
print(f"   - table_schemas.txt")
print(f"   - data_quality_issues.txt")
print(f"   - sample_data.csv")
print(f"   - current_gameweek_distribution.png")
print(f"   - historical_records_per_gw.png")
print(f"   - missing_data_trends.png")
print(f"\n🎯 Next: Review the report and proceed to Part 1B (Goals Analysis)")
print(f"\n📈 Data Quality Score: {ml_ready_score}/100 ({readiness_status} LIGHT)")
print(f"\n💡 Recommendation: {'Proceed with ML modeling' if ml_ready_score >= 80 else 'Review issues before proceeding'}")